In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [ ]:
# !pip install --upgrade datasets
# !pip install evaluate rouge_score
# import evaluate
# import datasets
# print(datasets.__version__)

In [ ]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict

import torch
import time
import os
import pandas as pd
import numpy as np

In [ ]:
from transformers import (
    GenerationConfig,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoModel,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    # pipeline,
    AutoConfig
)

### Paths

In [ ]:
# Model
fine_tune_path = '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM'
model_name = 'bart'
sub_model_name = 'bart_large_xsum_lora'
weights_path = f"{fine_tune_path}/{model_name}/{sub_model_name}/trained_model_9" # ADJUST
eval_path = f"{fine_tune_path}/{model_name}/{sub_model_name}/eval_{sub_model_name}.csv"

In [ ]:
# Dataset
mts_dialog_dir = "/content/drive/MyDrive/ClinicalNotesGen/Data/clinical_notes/data1_clinical_visit_note_summarization_corpus/data/mts-dialog"
mts_test_set_1_path = f"{mts_dialog_dir}/MTS_Dataset_Final_200_TestSet_1.csv"
mts_test_set_2_path = f"{mts_dialog_dir}/MTS_Dataset_Final_200_TestSet_2.csv"
mts_training_set_path = f"{mts_dialog_dir}/MTS_Dataset_TrainingSet.csv"
mts_val_set_path = f"{mts_dialog_dir}/MTS_Dataset_ValidationSet.csv"

aci_bench_dir = "/content/drive/MyDrive/ClinicalNotesGen/Data/clinical_notes/data1_clinical_visit_note_summarization_corpus/data/aci-bench"
challenge_data_dir = f"{aci_bench_dir}/challenge_data"
src_experiment_data_dir = f"{aci_bench_dir}/src_experiment_data"

### Dataset

In [ ]:
mts_training_set_df = pd.read_csv(mts_training_set_path, index_col='ID')
mts_val_set_df = pd.read_csv(mts_val_set_path, index_col='ID')
mts_test_set_1_df = pd.read_csv(mts_test_set_1_path, index_col='ID')
mts_test_set_2_df = pd.read_csv(mts_test_set_2_path, index_col='ID')

In [ ]:
# import all files in challenge_data_dir
challenge_data_files = os.listdir(challenge_data_dir)
challenge_data_dfs = {}
for file in challenge_data_files:
    challenge_data_dfs[file] = pd.read_csv(f"{challenge_data_dir}/{file}")

### Base model, tokenizer

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # bfloat16 for efficient memory usage
    # device_map="auto",
)

model.to(device)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
print_trainable_parameters(model)

trainable params: 8030261248 || all params: 8030261248 || trainable%: 100.0


### Check special tokens

In [ ]:
print("special_tokens_map:", tokenizer.special_tokens_map)
print("all_special_tokens:", tokenizer.all_special_tokens)
print("all_special_ids:   ", tokenizer.all_special_ids)
print("all_special_ids:   ", tokenizer.bos_token)
print("all_special_ids:   ", tokenizer.eos_token)

special_tokens_map: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|eot_id|>'}
all_special_tokens: ['<|begin_of_text|>', '<|eot_id|>']
all_special_ids:    [128000, 128009]
all_special_ids:    <|begin_of_text|>
all_special_ids:    <|eot_id|>


In [ ]:
config = AutoConfig.from_pretrained(model_id)
print("bos_token_id:", config.bos_token_id)
print("eos_token_id:", config.eos_token_id)
print("pad_token_id:", config.pad_token_id)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

bos_token_id: 128000
eos_token_id: 128009
pad_token_id: None


In [ ]:
print(tokenizer.chat_template)

{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}{% endif %}


### Construct prompts

In [ ]:
dialogue = """
Doctor: What brings you back into the clinic today, miss?
Patient: I came in for a refill of my blood pressure medicine.
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?
Patient: No.
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.
"""

system = "You are a medical assistant trained to generate clinical notes from doctor-patient dialogues."
instruction = "Generate a clinical summary based on this information."

In [ ]:
# 3
message = [
    {"role": "system", "content": f"{system}"},
    # {"role": "user", "content": f"{instruction}{dialogue}"},
    {"role": "user", "content": f"{dialogue}"},
]
print(tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a medical assistant trained to generate clinical notes from doctor-patient dialogues.<|eot_id|><|start_header_id|>user<|end_header_id|>

Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  
Patient: No. 
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.  
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
# 1
input_text_1 = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system}<|eot_id|><|start_header_id|>user<|end_header_id|>{dialogue}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{instruction}<|eot_id|>
"""

In [ ]:
# 2
input_text_2 = f"""
{tokenizer.bos_token}
SYSTEM:{system}
USER:{dialogue}
ASSISTANT:{instruction}
{tokenizer.eos_token}
"""

In [ ]:
# 3
input_text_3 = [
    {"role": "system", "content": f"{system}"},
    {"role": "user", "content": f"{dialogue}"},
    # {"role": "assistant", "content": f"{instruction}"},
]

In [ ]:
# 4
input_text_4 = [
    {"role": "system", "content": f"{system}"},
    {"role": "user", "content": f"{dialogue}"},
    {"role": "assistant", "content": f"{instruction}"},
]

- input_text_1: rec by chatgpt

In [ ]:
# Tokenize the input text
inputs_1 = tokenizer(input_text_1, return_tensors="pt", truncation=True, padding=True)

# Generate
with torch.no_grad():
    outputs_1 = model.generate(
        inputs_1["input_ids"].to(model.device),
        max_new_tokens=256,  # Adjust based on desired output length
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

# Decode
generated_text_1 = tokenizer.decode(outputs_1[0], skip_special_tokens=True)
print(generated_text_1)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



systemYou are a medical assistant trained to generate clinical notes from doctor-patient dialogues.user
Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  
Patient: No. 
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.  
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.
assistantGenerate a clinical summary based on this information.
**Clinical Summary**

Patient, a 76-year-old white female, presented to the clinic today for a refill of her blood pressure medication. She has a

- input_text_2: based on existing special tokens

In [ ]:
# Tokenize the input text
inputs_2 = tokenizer(input_text_2, return_tensors="pt", truncation=True, padding=True)

# Generate
with torch.no_grad():
    outputs_2 = model.generate(
        inputs_2["input_ids"].to(model.device),
        max_new_tokens=256,  # Adjust based on desired output length
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

# Decode and print
generated_text_2 = tokenizer.decode(outputs_2[0], skip_special_tokens=True)
print(generated_text_2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




SYSTEM:You are a medical assistant trained to generate clinical notes from doctor-patient dialogues.
USER:
Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  
Patient: No. 
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.  
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.

ASSISTANT:Generate a clinical summary based on this information.

**Clinical Summary**

**Patient Information:**

* Patient's name: [Not provided]
* Age: 76
* Race: White
* Sex: Female

**Chief Complaint:

- input_text_3: same as on HF

In [ ]:
input_ids_3 = tokenizer.apply_chat_template(
    input_text_3,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs_3 = model.generate(
    input_ids_3,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response_3 = outputs_3[0][input_ids_3.shape[-1]:]
print(tokenizer.decode(response_3, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here is the clinical note:

**Patient Information:**

* Name: [Not specified]
* Date of Birth: [Not specified]
* Age: 76
* Race: White
* Sex: Female

**Chief Complaint:**

* Patient presented for a refill of blood pressure medicine.

**Past Medical History:**

* Hypertension
* Osteoarthritis
* Osteoporosis
* Hypothyroidism
* Allergic rhinitis
* Kidney stones

**Current Symptoms:**

* None reported

**Review of Systems:**

* Fever or chills: No
* Cough: No
* Congestion: No
* Nausea: No
* Vomiting: No
* Chest pain: No
* Chest pressure: No

**Assessment:**

* Patient's blood pressure medicine is due for refill.
* Patient's chronic medical conditions are stable, with no reported changes or concerns.

**Plan:**

* Refilled blood pressure medicine as prescribed.
* Patient will be seen again in [insert timeframe] for follow-up.
* Patient will continue to monitor and manage their chronic medical conditions as instructed.

**Next Steps:**

* Schedule follow-up appointment for [insert timeframe]

In [ ]:
input_ids_4 = tokenizer.apply_chat_template(
    input_text_4,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs_4 = model.generate(
    input_ids_4,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response_4 = outputs_4[0][input_ids_4.shape[-1]:]
print(tokenizer.decode(response_4, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


**Clinical Summary**

**Patient Information:**

* Name: (not provided)
* Age: 76 years old
* Race: White female

**Chief Complaint:**

* Request for refill of blood pressure medicine

**Medical History:**

* Hypertension
* Osteoarthritis
* Osteoporosis
* Hypothyroidism
* Allergic rhinitis
* Kidney stones

**Current Symptoms:**

* None reported

**Review of Systems:**

* Respiratory: No fever, chills, cough, congestion
* Gastrointestinal: No nausea, vomiting
* Cardiovascular: No chest pain, chest pressure
* Musculoskeletal: No new complaints related to osteoarthritis or osteoporosis

**Assessment:**

* Patient's chronic medical conditions are stable, with no new symptoms or concerns reported.
* Patient is due for a refill of blood pressure medicine.

**Plan:**

* Refill prescription for blood pressure medicine as needed.
* Schedule follow-up appointment to monitor patient's chronic medical conditions and address any new concerns.

**Next Steps:**

* Patient to return to clinic for follo

# 1) Original model

###  Zero Shot Inferencing
1. select a test example from dataset
2. create prompt
3. tokenize prompt
4. feed tokenized prompt to model
5. decode the output

In [ ]:
# Generation
generation_config = GenerationConfig(
    max_new_tokens=216,
    num_beams=1,
    do_sample=False,
    early_stopping=False
)

In [ ]:
def generate_response_base(dialogue, note):
    prompt = f"""{dialogue}"""

    # tokenize prompt
    input = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    input = {k: v.to(device) for k, v in input.items()}

    # Token length of note
    note_token_length = tokenizer(
        note,
        return_tensors='pt'
    )

    print(f"Input token length: {input['input_ids'].shape[1]}")
    print(f"Token length of note: {note_token_length['input_ids'].shape[1]}")

    # decode the output
    output = tokenizer.decode(
        model.generate(
            input_ids=input["input_ids"],
            generation_config=generation_config,
        )[0],
        skip_special_tokens=True
    )

    # Print results
    dash_line = '-'.join('' for x in range(100))
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'REFERENCE SUMMARY:\n{note}')
    print(dash_line)
    print(f'BASE MODEL:\n{output}')

In [ ]:
generate_response_base(mts_training_set_df.iloc[0]['dialogue'], mts_training_set_df.iloc[0]['section_text'])

Input token length: 512
Token length of note: 102
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  
Patient: No. 
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.  
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.
---------------------------------------------------------------------------------------------------
REFERENCE SUMMARY:
The patient is a 76-year-old wh

In [ ]:
generate_response_base(mts_training_set_df.iloc[30]['dialogue'], mts_training_set_df.iloc[30]['section_text'])

Input token length: 512
Token length of note: 15
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Doctor: Hi, what brings you here today?
Patient: I feel this lump on the upper side of my breast. 
Doctor: Both sides? 
Patient: No, only on the left side. 
Doctor: Do you feel it deep inside or on the outer side?
Patient: Only on the outer side.
---------------------------------------------------------------------------------------------------
REFERENCE SUMMARY:
Suspicious calcifications upper outer quadrant, left breast.
---------------------------------------------------------------------------------------------------
BASE MODEL:
A patient has described the moment she found out she had breast cancer.


In [ ]:
generate_response_base(challenge_data_dfs['train.csv'].iloc[0]["dialogue"], challenge_data_dfs['train.csv'].iloc[0]["note"])

Input token length: 512
Token length of note: 856
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
[doctor] hi , martha . how are you ?
[patient] i'm doing okay . how are you ?
[doctor] i'm doing okay . so , i know the nurse told you about dax . i'd like to tell dax a little bit about you , okay ?
[patient] okay .
[doctor] martha is a 50-year-old female with a past medical history significant for congestive heart failure , depression and hypertension who presents for her annual exam . so , martha , it's been a year since i've seen you . how are you doing ?
[patient] i'm doing well . i've been traveling a lot recently since things have , have gotten a bit lighter . and i got my , my vaccine , so i feel safer about traveling . i've been doing a lot of hiking . uh , went to washington last weekend to hike in northern cascades, like around the mount baker area .
[doctor] nice . that's great . i'm glad to hear that you're stay

# 2) Fine-tuning with LoRA

### Pre-process dataset
- 1. convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM
  + prepend an instruction to the start of the dialog with `Summarize the following conversation` and to the start of the summary with `Summary`
- 2. tokenize and pull out their `input_ids` (1 per token)

In [ ]:
mts_training_set_df = mts_training_set_df[["dialogue", "section_text"]]
mts_val_set_df = mts_val_set_df[["dialogue", "section_text"]]

train_ds = Dataset.from_pandas(mts_training_set_df)
val_ds = Dataset.from_pandas(mts_val_set_df)
dataset = DatasetDict({'train': train_ds, 'validation': val_ds})

In [ ]:
def tokenize_function(example):
    prompt = ["" + dialogue + "" for dialogue in example["dialogue"]]

    inputs = tokenizer(
        prompt,
        padding="longest", # max_length, longest
        truncation=True,
        # return_tensors="pt",
        max_length=512
    )

    targets = tokenizer(
        example["section_text"],
        padding="longest",
        truncation=True,
        # return_tensors="pt",
        max_length=256
    )

    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["dialogue", "section_text"])

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['ID', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1201
    })
    validation: Dataset({
        features: ['ID', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

## LoRA config

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
from transformers import EarlyStoppingCallback

In [ ]:
modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'down_proj', 'up_proj', 'lm_head']
available_modules = set()

for name, _ in model.named_modules():
    for module in modules:
        if module in name:
            available_modules.add(module)

# Convert to list and print
available_modules = list(available_modules)
for module in available_modules:
    print(module)

k_proj
q_proj
v_proj
lm_head


In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    target_modules = ['q_proj','k_proj','v_proj','lm_head'],
    r=64,
    lora_alpha=16,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)

# Print number of trainable params (should be a small fraction)
model.print_trainable_parameters()

trainable params: 17,438,208 || all params: 423,728,640 || trainable%: 4.1154


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [ ]:
model.print_trainable_parameters()

trainable params: 17,438,208 || all params: 423,728,640 || trainable%: 4.1154


## Fine-Tune
- pass the preprocessed dataset with reference to the original model to `Trainer` class

In [ ]:
from datetime import datetime, timedelta, timezone
utc_plus_7 = timezone(timedelta(hours=7))
now = datetime.now(utc_plus_7)
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

checkpoints_path = f"{weights_path}/dialogue-summary-training-{timestamp}"
print(checkpoints_path)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = TrainingArguments(
    output_dir=checkpoints_path,

    # Core training
    learning_rate=2e-5, # 2e-5, 5e-5
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2, # 1, 2
    num_train_epochs=100,

    # Evaluation & Checkpointing
    eval_strategy="steps", # epoch, steps
    eval_steps=20,

    save_strategy="steps", # epoch, steps
    save_steps=20, # Save checkpoint every n steps
    save_total_limit=1, # Keep only the last checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Logging
    logging_strategy="steps",
    logging_steps=4, # Log every n steps

    # Precision & Speed
    bf16=True, # better stability
    fp16=False,
    gradient_checkpointing=False, # Disabled for speed

    # Optimization
    lr_scheduler_type="cosine",
    # warmup_steps=50, # Reduced for faster learning start
    optim="adamw_torch",
    warmup_ratio=0.03,
    weight_decay=0.01,
    # max_grad_norm=1.0,

    # Reporting
    # predict_with_generate=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # label_names=["labels"],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01)]
)

/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/dialogue-summary-training-2025-05-30_09-37-43


<ipython-input-23-50d6d69f285c>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
20,9.990200,10.061129
40,10.005200,9.967956
60,9.752300,9.666766
80,9.323300,9.256977
100,8.937100,8.904963
120,8.608600,8.560840
140,8.167300,8.227370
160,7.980400,7.786277
180,7.427300,7.312186
200,6.874400,6.721805


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=740, training_loss=3.805638921260834, metrics={'train_runtime': 573.8162, 'train_samples_per_second': 209.3, 'train_steps_per_second': 3.311, 'total_flos': 5.320596065353728e+16, 'train_loss': 3.805638921260834, 'epoch': 38.94736842105263})

In [ ]:
weights_path

'/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9'

### Save the fine-tuned model

In [ ]:
# Save the trained model
trainer.save_model(weights_path)
# Save tokenizer
tokenizer.save_pretrained(weights_path)

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/tokenizer_config.json',
 '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/special_tokens_map.json',
 '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/vocab.json',
 '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/merges.txt',
 '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/added_tokens.json',
 '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/bart/bart_large_xsum_lora/trained_model_9/tokenizer.json')

# 3) Check fine-tuned model

### Load the fine-tuned model

In [ ]:
!ls -lh $weights_path

total 268M
-rw------- 1 root root  874 May 29 10:07 adapter_config.json
-rw------- 1 root root 263M May 29 10:07 adapter_model.safetensors
drwx------ 2 root root 4.0K May 29 10:02 dialogue-summary-training-2025-05-29_17-02-42
drwx------ 3 root root 4.0K May 29 10:06 dialogue-summary-training-2025-05-29_17-05-27
-rw------- 1 root root 446K May 29 10:07 merges.txt
-rw------- 1 root root 5.0K May 29 10:07 README.md
-rw------- 1 root root  279 May 29 10:07 special_tokens_map.json
-rw------- 1 root root 1.3K May 29 10:07 tokenizer_config.json
-rw------- 1 root root 3.4M May 29 10:07 tokenizer.json
-rw------- 1 root root 5.5K May 29 10:07 training_args.bin
-rw------- 1 root root 780K May 29 10:07 vocab.json


In [ ]:
# Load the fine-tuned model
# base_model = AutoModelForSeq2SeqLM.from_pretrained(weights_path).to(device)
# tokenizer = AutoTokenizer.from_pretrained(weights_path)

In [ ]:
config = PeftConfig.from_pretrained(weights_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
base_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, return_dict=True, device_map='auto')
base_model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
ft_model = PeftModel.from_pretrained(base_model, weights_path).to(device)
ft_model.eval()
print_trainable_parameters(base_model)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


trainable params: 0 || all params: 423728640 || trainable%: 0.0


In [ ]:
print_trainable_parameters(ft_model)

trainable params: 0 || all params: 423728640 || trainable%: 0.0


### Compare weights

In [ ]:
# compare final layer weights
base_weights = base_model.lm_head.weight
ft_weights = ft_model.lm_head.weight

diff = torch.norm(ft_weights - base_weights)
print(f"Weight difference: {diff.item()}")

Weight difference: 0.0


### Responses from base and fine-tuned model

In [ ]:
def generate_response(dialogue, note):
    prompt = f"""{dialogue}"""

    # tokenize prompt
    input = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    input = {k: v.to(device) for k, v in input.items()}

    # Token length of note
    note_token_length = tokenizer(
        note,
        return_tensors='pt'
    )

    print(f"Input token length: {input['input_ids'].shape[1]}")
    print(f"Token length of note: {note_token_length['input_ids'].shape[1]}")

    # decode the output
    output = tokenizer.decode(
        base_model.generate(
            input_ids=input["input_ids"],
            generation_config=generation_config,
        )[0],
        skip_special_tokens=True
    )

    output_2 = tokenizer.decode(
        ft_model.generate(
            input_ids=input["input_ids"],
            generation_config=generation_config,
        )[0],
        skip_special_tokens=True
    )

    # Print results
    dash_line = '-'.join('' for x in range(100))
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'REFERENCE SUMMARY:\n{note}')
    print(dash_line)
    print(f'BASE MODEL:\n{output}')
    print(dash_line)
    print(f'FINE-TUNED MODEL:\n{output_2}')

In [ ]:
generate_response(mts_training_set_df.iloc[30]['dialogue'], mts_training_set_df.iloc[30]['section_text'])

Input token length: 512
Token length of note: 15
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Doctor: Hi, what brings you here today?
Patient: I feel this lump on the upper side of my breast. 
Doctor: Both sides? 
Patient: No, only on the left side. 
Doctor: Do you feel it deep inside or on the outer side?
Patient: Only on the outer side.
---------------------------------------------------------------------------------------------------
REFERENCE SUMMARY:
Suspicious calcifications upper outer quadrant, left breast.
---------------------------------------------------------------------------------------------------
BASE MODEL:
A patient has described the moment she found a lump on her breast in a hospital examination.
---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL:
A patient has described the moment she found a lump on her breast in a hospital examina

In [ ]:
generate_response(mts_training_set_df.iloc[12]['dialogue'], mts_training_set_df.iloc[12]['section_text'])

Input token length: 512
Token length of note: 4
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Doctor:  Are you talking any medications?
Patient: No, I don't believe in meds.
Doctor: Oh really, let me check your list.
Patient: Yeah never.
Doctor: Wow amazing.
---------------------------------------------------------------------------------------------------
REFERENCE SUMMARY:
None.
---------------------------------------------------------------------------------------------------
BASE MODEL:

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL:

